In [ ]:
# import data
import pandas as pd
import cv2

data = []

category = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}

# get text data
with open ("/kaggle/input/garbage-classification/zero-indexed-files.txt", "r") as text:
    data = text.read().splitlines()
    

df = pd.DataFrame(columns=['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash'])

for img_pair in data[:10]:
    print(img_pair)
    path = f"/kaggle/input/garbage-classification/Garbage classification/Garbage classification/{category[img_pair[len(img_pair) - 1]]}/{img_pair[:-2]}"
    print(path)
    img = cv2.imread(path)
    print(img)
    
    


In [ ]:
!pip install flickrapi

In [ ]:
import flickrapi
import xml.etree.ElementTree as ET
import urllib.request
import os

In [ ]:
# open the .txt list of bird species and read them in as a list 
with open("/kaggle/input/recyclableitemslabels2/recyclable-items version 2.txt") as recyclable_items_file:
    recyclable_items = [line.strip() for line in recyclable_items_file]

In [ ]:
#get an api key and secret from flickr
api_key = u'150adfe3c2bb6b37e2160d88724730c6'
api_secret = u'ec7d59b70b6720a6'
flickr = flickrapi.FlickrAPI(api_key, api_secret)

In [ ]:
from tqdm import tqdm
for item in recyclable_items:
    photos = flickr.walk(text=item,
                        tag_mode='all',
                        tags=item,
                        extras='url_c',
                        privacy_filters = 1,
                        per_page=500,         
                        sort='relevance')
    for i, photo in tqdm(enumerate(photos)):
        url = photo.get('url_c')
        #if an error occurs just keep moving
        try:
            # first 10 images are for test
            if i <= 19:
                TestDirectory = f'/kaggle/working/ITEMS/test/{item}'
                #check if directory exists
                if not os.path.exists(TestDirectory):
                    os.makedirs(TestDirectory)
                #create file path
                filepath = os.path.join(TestDirectory, f'{i+1}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)

            #next five images are for validation after model has been created
            elif i < 25 and i > 19:
                #create a validation folder in another location
                ValidateDirectory = f'/kaggle/working/Item_Validate/validate/{item}'
                #check if directory exists
                if not os.path.exists(ValidateDirectory):
                    os.makedirs(ValidateDirectory)
                #create file path
                filepath = os.path.join(ValidateDirectory, f'{i+1}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)

            #stop after 200 total images
            elif i > 500:
                break

            #all remaining images go to training
            else:
                TrainDirectory = f'/kaggle/working/ITEMS/train/{item}'
                #check if directory exists
                if not os.path.exists(TrainDirectory):
                    os.makedirs(TrainDirectory)
                #create file path    
                filepath = os.path.join(TrainDirectory, f'{i-9}.jpg')
                # Download image from the url and save it to '00001.jpg'
                urllib.request.urlretrieve(url, filepath)
        except:
            pass
import shutil
shutil.make_archive("items", 'zip', "/kaggle/working/ITEMS")

In [ ]:
import shutil
shutil.make_archive("validation", 'zip', "/kaggle/working/Item_Validate/validate/")

In [ ]:
import shutil
shutil.rmtree('/kaggle/working/')

# Creating Dataset

In [1]:
import tensorflow as tf

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "/kaggle/input/ecycle-images/E Cycle Image Data/recyclable items/train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=True,
)



Found 8917 files belonging to 27 classes.


2023-02-08 01:59:04.510399: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 01:59:04.514199: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 01:59:04.710009: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 01:59:04.711404: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-02-08 01:59:04.712573: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [5]:
import tensorflow_datasets as tfds
ds_numpy = tfds.as_numpy(train_dataset)